<a href="https://colab.research.google.com/github/lautalom/diploUNC/blob/main/Cifrados_bloques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Falsificacion en modo ECB
[Enunciado](https://ciberseguridad.diplomatura.unc.edu.ar/cripto/doc/ecb-forge.html)


En este desafío, el servidor permite a un usuario registrarse con un correo electrónico, y devuelve un perfil, opcionalmente cifrado con AES en modo ECB.

Si el usuario registró el correo usuario@example.com , el perfil tendrá la forma
user=usuario@example.com&id=547&role=user

Es decir, está compuesto por un conjunto de pares atributo-valor, de la forma <atributo>=<valor>, y separados el carácter &, de la misma forma que en la query string de una URL.

Para obtener el perfil, se debe hacer una peticion GET a una URL de la forma:

https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-forge/<email\>/register?email=\<email-a-registrar>
        
dónde <email> debe ser reemplazado por una dirección de correo electrónico registrada, y <email-a-registrar> por la dirección de correo electrónico que se desee registrar.

En el siguiente ejemplo, ejecutado con curl, la dirección que está operando el desafío es user@example.com y la dirección que se desea registrar es usuario@ejemplo.edu.ar:

```
$ curl https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-forge/user@example.com/register?email=usuario@ejemplo.edu.ar
user=usuario@ejemplo.edu.ar&id=207&role=user
```

Si se le pasa un segundo par encrypted=true en la consulta, el servidor devuelve el mismo perfil, cifrado con AES en modo ECB, y codificado en base64.

Veamos el mismo ejemplo:

```
$ curl 'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-forge/user@example.com/register?email=usuario@ejemplo.edu.ar&encrypted=true'
```
El servidor devuelve el perfil cifrado y codificado en base64. Obsérvese que hemos encerrado todo el argumento de curl entre comillas para evitar que el shell interprete el carácter &. Obviamente esta precaución es innecesaria si se utiliza un browser.

El desafío es alterar el mensaje cifrado de manera tal que al descifrarlo incluya el par role=admin. Esta alteración es posible porque el modo ECB es maleable. Es posible intercambiar o insertar bloques en el texto cifrado de manera tal que al descifrarlo el texto claro haya sido modificado de manera predecible.

# Análisis
La clave para resolver este desafío es comprender que podemos armar un mensaje falsificado combinando bloques que contienen texto seleccionado por nosotros. En este caso, el único texto bajo nuestro control es el correo electrónico, por lo que será necesario enviar distintas direcciones de manera tal que ciertos bloques contengan el texto que nosotros deseamos.

# Puntos a tener en cuenta
- El algoritmo utilizado es AES, por lo que los bloques son de 128 bits (16 bytes).
- El texto cifrado enviado como respuesta debe descifrarse a una consulta (query string) válida:
- Debe tener exactamente un campo user, con una dirección de correo válida.
- Debe contener al menos un campo id.
- Debe contener al menos un campo role con el valor admin.
- Puede contener otros pares atributo-valor aparte de los mencionados.
- Como el modo utilizado es ECB, se utiliza relleno (padding). El relleno utilizado es el estandarizado en PKCS7: se aplican tantos bytes como sea necesario para que el mensaje sea múltiplo del tamaño de bloque, y el valor de esos bytes es la longitud del relleno. Si el mensaje original era múltiplo del tamaño de bloque, el relleno es un bloque adicional con todos sus bytes con el valor 16. Al descifrar el mensaje, el relleno debe ser válido.

In [ ]:
import requests
from base64 import b64decode as dec
from base64 import b64encode as enc

1. register a user that ends a block with 'role='
2. register another user that starts block with 'admin&id=....'
3. put the admin block besides the role= of the first user

note: the cleartext includes the whole formatting, not just the email.

In [ ]:
mailex = 'hellokitty@mail.com'
email = "lautarolombardi19@gmail.com"

def makereq(address, encode=0):
  url = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-forge/{email}/register?email={address}"
  if encode:
    url += '&encrypted=true'
  request = requests.get(url)
  return request.text

In [ ]:
user1, user2, user3 = '', '', ''
first = len(makereq(mailex, 0))
user1, user2, user3 = '', '', ''
# register user that ends with role=, aka len==(multiple of 16)+4
for i in range(16):
  attempt = makereq(mailex + 'b'*i, 0)
  # register user that ends with role=, aka len==(multiple of 16)+4
  if len(attempt)%16-4==0:
    user1 = i

user1 = dec(makereq(mailex + user1 * 'b', 1))
user2 = mailex + (32-len('user=')-len(mailex)) * 'b' + 'admin'
# should be 5 places after newblock
print(makereq(user2, 0).find('&id'))
# should be exactly at newblock index
if makereq(user2, 0).find('admin')%16 == 0:
  user2 = dec(makereq(user2, 1))

37


In [ ]:
# take the admin block out of user2
admin_cipher = user2[-16*2:-16]
new_user = user1[:-16] + admin_cipher + user1[-16:]

In [ ]:
ans = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-forge/{email}/answer"
request = requests.post(url=ans, files={'message': enc(new_user)})
print(request.text)

¡Ganaste!



# Cambio de bits en modo CBC
En este desafío, el servidor permite a un usuario registrarse con un correo electrónico y un conjunto de datos adicionales, y devuelve un perfil cifrado con AES en modo CBC.

Si el usuario registró el correo usuario@example.com y puso como datos “Juan P. Usuario”, el perfil tendrá la forma:

user=usuario@example.com;data=Juan P. Usuario;role=user

Es decir, está compuesto por un conjunto de pares atributo-valor, de la forma \<atributo>=\<valor>, y separados por punto y coma.

El servidor devuelve el perfil cifrado, precedido por un IV de 16 bytes, y codificado en base64.

El desafío es alterar el mensaje cifrado de manera tal que al descifrarlo incluya el par role=admin. Esta alteración es posible porque el modo CBC es maleable. Es posible modificar el texto cifrado de manera tal que al descifrarlo el texto claro haya sido modificado de manera (parcialmente) predecible.

Este tipo de ataque se denomina bit flipping, porque el cambio de un bit en el texto cifrado provoca un cambio el bit correspondiente del siguiente bloque de texto claro (aunque altera completamente el texto claro del bloque modificado).

Para obtener los textos cifrados, se debe hacer un requerimiento POST a una URL de la forma:

https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-bitflip/<email\>/register
        
dónde \<email> debe ser reemplazado por una dirección de correo electrónico registrada.

El contenido debe ser de tipo FORM, con dos campos:

- Un campo email, con una dirección de correo a registrar. No es necesario que sea la misma dirección utilizada para acceder al desafío.
- Un campo data, con datos arbitrarios, codificado en base64. Los datos no pueden contener el carácter ‘;’ ni el carácter ‘=’.

La respuesta será un perfil como el descripto, cifrado, concatenado con el IV y codificado en base64.

Si en lugar de hacer un requerimiento POST se hace un requerimiento GET, muestra un formulario que permite cargar los campos requeridos.

# Analisis
La clave para resolver este desafío consiste en entender el comportamiento del modo CBC ante modificaciones del texto cifrado. El resultado de descifrar el bloque modificado es impredecible, pero el siguiente bloque será modificado en exactamento los mismos bits que fueron cambiados. Como en este caso el contenido del campo data es arbitrario, cualquier modificación de su contenido no será detectada, por lo que la solución es escoger un valor para data que tenga la longitud adecuada, de manera que sepamos en qué bloque están los datos a modificar, y alterar el bloque anterior de manera apropiada.

## Puntos a tener en cuenta
El algoritmo utilizado es AES, por lo que los bloques son de 128 bits (16 bytes).
El texto cifrado enviado como respuesta debe descifrarse a una consulta válida, con pares atributo-valor separados por punto y coma:
Debe tener exactamente un campo user, con una dirección de correo válida.
Debe tener exactamente un campo role con el valor admin.
Como el modo utilizado es CBC, se utiliza relleno (padding). El relleno utilizado es el estandarizado en PKCS7: se aplican tantos bytes como sea necesario para que el mensaje sea múltiplo del tamaño de bloque, y el valor de esos bytes es la longitud del relleno. Si el mensaje original era múltiplo del tamaño de bloque, el relleno es un bloque adicional con todos sus bytes con el valor 16. Al descifrar el mensaje, el relleno debe ser válido.

In [ ]:
import base64
import requests

In [ ]:
fakemail = 'hellokitty@mail.com'
email = "lautarolombardi19@gmail.com"
def makereq(data):
  challenge_url = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-bitflip/{email}/register"
  request = requests.post(challenge_url,
                          files={'email': fakemail,
                                 'data': base64.b64encode(bytes(data, 'ascii'))
                                 }
                          )
  return request.text

# como funciona CBC descifrando
toma el ciphertext de un bloque y se lo xorea al descifrado del bloque siguiente para obtener el texto plano de ese bloque siguiente. Con lo cual, se puede tocar el ciphertext del bloque n para alterar el texto plano del bloque n+1.

In [ ]:
def closest_multiple(m, k):
    remainder = m % k
    diff = k - remainder

    if diff <= remainder:
        closest = m + diff
    else:
        closest = m - remainder
    return closest

# armamos el payload data
current_lengths = len('user=')  + len(fakemail) + len(';data=')
# this fills previous to last block with trash and first byte of last block with 'b'
data = 'b' * (48-current_lengths) + 'b'
encrypted = base64.b64decode(makereq(data))
print(len(encrypted))
array = bytearray(encrypted)

80


en el ultimo bloque tenemos 'b;role=user' + padding.
Tenemos que cambiar toda la substring inicial por ';role=admin', rompiendo el ciphertext del bloque anterior adecuadamente.

Sabemos que el XOR de array[-16\*2:-16] con el decrypt(array[-16:]) nos da el texto claro ';role=user' + padding. Con lo cual, hay que cambiar los bytes iniciales de array[-16\*2:-16] tal que una vez xoreados contra lo que decrypt(array[-16:]) devuelva el texto ';role=admin'

In [ ]:
target_string = ';role=admin'
source_string = 'b;role=user'
def find_start_bytes():
  result = []
  for i in range(len(source_string)):
    # rationale: array[-32] XOR decrypt(array[-16]) = 'b', what is decrypt?
    decrypted_byte = array[-32+i] ^ ord(source_string[i])
    # rationale: something XOR decrypt(array[-16])=';', what is something?
    new_byte = ord(target_string[i]) ^ decrypted_byte
    result.append(new_byte)
  return bytes(result)

start_bytes = find_start_bytes()
new_cipher= bytes(array[:-16*2]) + start_bytes + bytes(array[-16*2+len(start_bytes):])
print(len(new_cipher)==len(encrypted))

True
b'\n\x94#\xf4g\x97\xb0\xc9\xbb\xb3=\xd7s\xa7t\xba\x04\xc5\xe0:\\\xfd\x93\xb9`q$\x88\xdcf\xbf\xe1'
b'\n\x94#\xf4g\x97\xb0\xc9\xbb\xb3='


In [ ]:
ans = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-bitflip/{email}/answer"
request = requests.post(url=ans, files={'message': base64.b64encode(new_cipher)})
print(request.text)

¡Ganaste!



# Descifrado en modo ECB
En este desafío, el servidor recibe un mensaje elegido por el usuario, codificado en base64, concatena un mensaje secreto con ese mensaje, y devuelve el resultado cifrado con AES en modo ECB y codificado en base64.

Es decir, si \<mensaje> es el texto elegido por el usuario, el servidor enviará:

encode_base64(AES-ECB(decode_base64(\<mensaje>)||\<mensaje_secreto>))
donde:

- encode_base64() y decode_base64() denotan la codificación y decodificación en base64, respectivamente.
- AES-ECB() denota la aplicación de AES en modo ECB
- \<mensaje_secreto> es un mensaje generado por el servidor.
- || denota concatenación.

El desafío consiste en descifrar el mensaje secreto.

Para obtener el texto cifrado, se debe hacer un requerimiento POST a una URL de la forma:

https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-decrypt/<email\>/encrypt
        
dónde \<email> debe ser reemplazado por una dirección de correo electrónico registrada.

El contenido debe ser de tipo FORM, con un campo message que contiene el mensaje elegido, codificado en base64.

La respuesta será un texto cifrado codificado en base64, como se describió anteriormente.

Si en lugar de hacer un requerimiento POST se hace un requerimiento GET, muestra un formulario que permite cargar el campo requerido.

# Análisis
La clave para resolver este desafío consiste en comprender que se pueden enviar distintos textos elegidos, y analizar lo respondido por el servidor.

Una posible secuencia de pasos para resolver el desafío puede ser la siguiente:

- Determinar la longitud del mensaje secreto. La longitud del texto cifrado será igual a la longitud del texto claro elegido, más la longitud del mensaje secreto, más la longitud del relleno. Para ello podemos enviar primero un mensaje vacío, luego uno de longitud 1, luego otro de longitud 2, etc. Cuando cambie la longitud del texto cifrado será porque la longitud de nuestro mensaje, sumada a la longitud del mensaje secreto, completó un múltiplo del tamaño de bloque, y habremos averiguado la longitud deseada.
- Enviar un mensaje de 15 bytes. Esto significa que el primer bloque estará conformado por los 15 bytes que conocemos, y el primer byte del mensaje secreto, que no conocemos. Extraer el primer bloque del texto cifrado.
- Enviar sucesivos mensajes de 16 bytes. Los primeros 15 bytes deben ser iguales a los 15 bytes del paso anterior. Variar el último byte hasta que el primer bloque del texto cifrado sea igual al obtenido en el paso anterior. El primer byte del mensaje secreto será, pues, el byte utilizado como último byte del texto claro correspondiente a ese bloque.
- Repetir el paso 2, pero ahora enviando un bloque en el que el byte 15 es el byte que hemos encontrado.
- Repetir el paso 3 para averiguar el segundo byte del mensaje secreto.
- Descifrar el resto de los bytes del mensaje secreto de manera similar.

In [ ]:
import base64
import requests
import string

In [ ]:
email = "lautarolombardi19@gmail.com"
server = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-decrypt/{email}/encrypt"

def sendmessage_ecb(message):
  req = requests.post(server,files={'message': base64.b64encode(bytes(message, 'ascii'))})
  return base64.b64decode(req.text)

In [ ]:
def findBlockLength():
  message = ''
  start = len(sendmessage_ecb(message))
  for i in range(1,17):
    message = 'a' * i
    resp = sendmessage_ecb(message)
    if len(resp) > start:
      return (message, len(resp))

payload, msglength = findBlockLength()

In [ ]:
print(len(payload), msglength)
# we got an entire block of padding and some bytes of clear text
msglength = msglength - len(payload) - 16

16 848


In [ ]:
def bruteforce(target, source, decrypted):
  charset =  string.whitespace + string.ascii_lowercase + string.ascii_uppercase + string.digits + string.punctuation
  s1, s2 = len(target), len(decrypted)
  for i in charset:
    resp = sendmessage_ecb(target+decrypted+i)
    if resp[s1+s2-15:s1+s2+1] == source:
      return i
  return 'f'

def decryptme(secret_length, initial_message=''):
  decrypt = initial_message
  initial = 'a' * (secret_length-len(decrypt)-1)
  while len(initial):
    if len(initial)%16==0:
      print(decrypt)
    cipher = sendmessage_ecb(initial)
    new = bruteforce(initial, cipher[secret_length-16:secret_length], decrypt)
    decrypt += new
    initial = initial[1:]
  return decrypt

initialmsg2 = """...we must counterpose the overwhelming judgment provided by consistent
observations and inferences by the thousands.  The earth is billions of
years old and its living creatures are linked by ties of evolutionary
descent.  Scientists stand accused of promoting dogma by so stating, but
do we brand people illiberal when they proclaim that the earth is neither
flat nor at the center of the universe?  Science *has* taught us some
things with confidence!  Evolution on an ancient earth is as well
established as our planet's shape and position.  Our continuing struggle
to understand how evolution happens (the "theory of evolution") does not
cast our documentation of its occurrence -- the "fact of evolution" --
into"""
decrypted = decryptme(msglength, initialmsg2)

print(decrypted)

...we must counterpose the overwhelming judgment provided by consistent
observations and inferences by the thousands.  The earth is billions of
years old and its living creatures are linked by ties of evolutionary
descent.  Scientists stand accused of promoting dogma by so stating, but
do we brand people illiberal when they proclaim that the earth is neither
flat nor at the center of the universe?  Science *has* taught us some
things with confidence!  Evolution on an ancient earth is as well
established as our planet's shape and position.  Our continuing struggle
to understand how evolution happens (the "theory of evolution") does not
cast our documentation of its occurrence -- the "fact of evolution" --
into 
...we must counterpose the overwhelming judgment provided by consistent
observations and inferences by the thousands.  The earth is billions of
years old and its living creatures are linked by ties of evolutionary
descent.  Scientists stand accused of promoting dogma by so statin

In [ ]:
print(len(decrypted))
answer = f'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-decrypt/{email}/answer'
send = requests.post(answer,files={'message': decrypted})
print(send.text)

815
Lo siento, siga participando.



# Padding oracle

En este desafío, el servidor produce un mensaje cifrado con AES en modo CBC y codificado en base64. El IV es enviado como primer bloque del texto cifrado.

El desafío consiste en descifrar el mensaje secreto.

Si se realiza un requerimiento POST a una url de la forma:

https://ciberseguridad.diplomatura.unc.edu.ar/cripto/padding-oracle/<email\>/decrypt
con contenido de tipo FORM, y un campo message con un texto cifrado codificado en base64, el servidor responderá:

- OK, con código 200, si puede descifrar.
- el mensaje no ha sido codificado en base64, con código 400 (Bad Request), si no está correctamente codificado en base64.
- Bad padding bytes, con código 400 (Bad Request), si el relleno es incorrecto o el tamaño del mensaje decodificado no es múltiplo del tamaño de bloque.

# Analisis

Utilizaremos la siguiente notación:

- $P_i$ es el $i$-ésimo bloque del texto claro.
- $C_i$ es el $i$-ésimo bloque del texto cifrado.
- $P_i = D_k(C_i) \oplus C_{i-1}$
- Llamaremos $D_i$ a $D_k(C_i)$, es decir, al resultado de aplicar la función de descifrado sobre un bloque de texto cifrado.
- Llamaremos $B_{i,j} $ al $j$-esimo byte del bloque $i$,donde puede $B$ ser $C$, $P$ o $D$ . Por ejemplo, $C_{1,16}$ es el último byte del primer bloque del texto cifrado.

Una posible secuencia de pasos para resolver el desafío puede ser la siguiente:

1. Enviar a https://ciberseguridad.diplomatura.unc.edu.ar/cripto/padding-oracle/<email\>/decrypt los dos primeros bloques del texto cifrado. Cuando el servidor lo descifre, pueden pasar dos cosas:
  - El resultado tiene un relleno válido. Esto significa que lo más probable es que el texto descifrado termine con un 1, indicando que existe un único byte de relleno. En este caso el servidor devolverá un código 200 con el mensaje **OK**.
  - Si el resultado de descifrar no genera un relleno válido, retornará un error 400 con el texto **Bad padding bytes**. Esta opción es la más probable. En ese caso cambiamos el último byte del primer bloque y repetimos el intento hasta que obtenemos un **OK**.
2. Ahora hemos obtenido un primer bloque $C'_1$ tal que produce un 1 en el último byte del segundo bloque: $D_{2,16} \oplus C'_{1,16} = 1$. Pero sabemos que $P_{2,16} = D_{2,16} \oplus C_{1,16}$, por lo que $P_{2,16} = 1 \oplus C'_{1,16} \oplus C_{1,16}$. Hemos averiguado el último byte del segundo bloque enviado.
3. Como conocemos su valor, podemos manipular el último byte del primer bloque para que ahora descifre al valor 2, y repetimos el procedimiento, cambiando el penúltimo byte del primer bloque. Cuando obtengamos **OK**, lo más probable será que hayamos producido un relleno con los últimos dos bytes con el valor 2.
4. Repetimos el procedimiento hasta descifrar todo el bloque, y luego repetimos con todos los bloques.

Con un máximo de 4096 intentos por bloque, debería ser posible descifrar el mensaje completo.

In [ ]:
import base64
import requests
import string

In [ ]:
email = "lautarolombardi19@gmail.com"
server = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/padding-oracle/{email}/decrypt"
challenge = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/padding-oracle/{email}/challenge"

def get_cipher():
  req = requests.get(challenge)
  return base64.b64decode(req.text)

def bad_padding(message)->bool:
  req = requests.post(server,files={'message': base64.b64encode(message)})
  return 'OK' not in req.text

In [ ]:
cipher = get_cipher()
print(cipher)
BLOCK_SIZE = 16 # AES-128
# https://github.com/TheCrowned/padding-oracle-attack/blob/master/attack.py
def attack(ciphertext, bsize=BLOCK_SIZE):
    final = b''
    blocks = len(ciphertext)//bsize
    charset =  set(string.whitespace + string.ascii_lowercase + string.ascii_uppercase + string.digits + string.punctuation)
    for block_n in range(blocks-1):
        guessed_clear = b''
        # take first 2 blocks, then first 3,...
        split_ciphertext = ciphertext[block_n*bsize:block_n*bsize+bsize*2]
        #output of block cipher decoding values
        decoded_bytes = b'?' * BLOCK_SIZE
        for byte in range(bsize-1, -1, -1):
            new_pad_len = bsize - byte #(1, 2, 3...)

            #Build hacked ciphertext tail with values to obtain desired padding
            hacked_ciphertext_tail = b''
            for padder_index in range(1, new_pad_len):
                hacked_ciphertext_tail += bytes([new_pad_len ^ decoded_bytes[byte + padder_index]])

            for i in range(256):
                attack_str =  bytes([i ^ split_ciphertext[byte]]) #take last byte from iv, then second to last, then third to last
                if not (32 <= (i ^ new_pad_len) <= 126): #if not printable character,
                    continue
                hacked_ciphertext = split_ciphertext[:byte] + attack_str + hacked_ciphertext_tail + split_ciphertext[byte + 1 + new_pad_len - 1:]
                if not bad_padding(hacked_ciphertext):
                    # C'_{i,j} (es hacked_[byte]) ^ d_{i,j} = new_pad_len => C'_{i,j} ^  new_pad_len  = d_{i,j}
                    decoded_bytes = decoded_bytes[:byte] + bytes([hacked_ciphertext[byte] ^ new_pad_len]) + decoded_bytes[byte + 1:]
                    # i = C'_{i,j} ^ C_{i,j} por construccion de la attack string
                    guessed_clear = bytes([i^ new_pad_len]) + guessed_clear
                    break
        final += guessed_clear
        print(final)
    return guessed_clear
msg = attack(cipher)
print(msg)

b'\xf2\xa0mP\x7fF\x82\x85/\xf9\xd9h\x07F\x96\xed\xd3\xa1D\r\xee\xa1u9\x8c\xd9\xd8\xc6\xa3\x08g\xce\x0b\xfc7\xacC\x9ce"g\xab\x96\x18\xf0~`\xfa\xf3\xfdD\x94n\xa9\xc3j\xaf\xe1\x0f\xaa\xc3\xa2\x99\xba\x9e\x18+&5\x9f\xc0\x86q>n\x8e\xa0\xe3\x1e\x13\x04\x0b\xb0H\xb4\\\xd9,\x993\x19\xb0\x1c\xef\x08\x04\xdb\xc9\xa0\x95\x9a\x91F\xd3\xde\xfaB\x12\xcaoKi\x84\xfcc\xe6{\x87Q)\x0f\x9d\x1d\x99\xe7\xe0\x9b\xa4\x92\x99\xf6\xd3{}S\xf3\xfc\x904\xe1\xf8\x84e\xdf\x13\t\xf43#d*Cy\xf59\xd8\xe0\xc2$C\xab\xbc\xbd!\xdcL\xdc]:\xec.5{S\x173T1Lg\xa81T\x95B\xb3Z\x9bXzv\xce\x81\xcbj\xf2\xd1\xbc;\xba\x02\xec>4O\xb8\x90\xb8\xe3f9G\xfb\x82\xe4x\xcc\x96\xc2F\xc4~\xed=e\xefop\x9c\x8a\x84\xb4`\xaf\xfe\x1c\x11\xd0\xa2\r4\xc5}.\xc4\xf0\x12\xa0\x1e{\x0e\xcd\xdb\xed\x88\x81\x8d\x0e\xa3~\xd3\x0c\x1e\x84\xe2\x9d\xadE\x0b\xfdn\xb1\x05\x07_\xb0\xebA\xcc \xc6\xe3s\x97\x0f\x02\x87{A\xba\x1a\x0c\xf5B\xe0"\xbb2}\xc9Z<\xe1{\xadX{\'\x052\xfe\x94w\x80\x93\xe1MGFb\x19\xbb\xc1\x99!,\x97\xd9\xcb\xd4k\xd6\x16\x9am5\xb7\xa8\xcem{\x08S\xce\x04

NameError: ignored

In [ ]:
import base64
import requests
import time
import threading

email = "lautarolombardi19@gmail.com"
server = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/padding-oracle/{email}/decrypt"
challenge = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/padding-oracle/{email}/challenge"

def get_cipher():
  req = requests.get(challenge)
  return base64.b64decode(req.text)

def bad_padding(message)->bool:
  req = requests.post(server,files={'message': base64.b64encode(message)})
  return 'OK' not in req.text

cipher = get_cipher()
print(cipher)
BLOCK_SIZE = 16  # AES-128

def worker(block, results_dict, lock):
    bsize = BLOCK_SIZE
    final = b''
    # take first 2 blocks, then first 3,...
    split_ciphertext = ciphertext[block*bsize:block*bsize+bsize*2]
    #output of block cipher decoding values
    decoded_bytes = b'?' * BLOCK_SIZE
    print(f'Block {block} started')
    for byte in range(bsize-1, -1, -1):
        new_pad_len = bsize - byte #(1, 2, 3...)

        #Build hacked ciphertext tail with values to obtain desired padding
        hacked_ciphertext_tail = b''
        for padder_index in range(1, new_pad_len):
            hacked_ciphertext_tail += bytes([new_pad_len ^ decoded_bytes[byte + padder_index]])

        guessed_clear = b''
        for i in range(256):
            attack_str =  bytes([i ^ split_ciphertext[byte]]) #take last byte from iv, then second to last, then third to last
            if not (32 <= (i ^ new_pad_len) <= 126): #if not printable character,
                continue
            hacked_ciphertext = split_ciphertext[:byte] + attack_str + hacked_ciphertext_tail + split_ciphertext[byte + 1 + new_pad_len - 1:]
            if not bad_padding(hacked_ciphertext):
                # C'_{i,j} (es hacked_[byte]) ^ d_{i,j} = new_pad_len => d_{i,j} = C'_{i,j} ^ new_pad_len
                decoded_bytes = decoded_bytes[:byte] + bytes([hacked_ciphertext[byte] ^ new_pad_len]) + decoded_bytes[byte + 1:]
                # i = C'_{i,j} ^ C_{i,j} por construccion de la attack string
                guessed_clear = bytes([i^ new_pad_len]) + guessed_clear
                break
        final += guessed_clear
    with lock:
        results_dict[block] = final[::-1]

def format_time(seconds):
    minutes = int(seconds // 60)
    seconds %= 60
    return f"{minutes} minutes, {seconds:.2f} seconds"


def parallel_attack(ciphertext, bsize=BLOCK_SIZE):
    # Split the ciphertext into blocks for each worker
    blocks = len(ciphertext) // bsize
    block_ranges = [i for i in range(blocks - 1)]  # Define block ranges for each worker

    # Create a dictionary to store results
    results_dict = {}

    lock = threading.Lock()
    start_time = time.time()
    # Create threads for each block range
    threads = []
    for block_range in block_ranges:
        t = threading.Thread(target=worker, args=(block_range, results_dict, lock))
        threads.append(t)
        t.start()

    # Wait for all threads to complete
    for t in threads:
        t.join()
    # Measure the end time for the entire process
    end_time = time.time()

    # Calculate the total elapsed time
    total_elapsed_time = end_time - start_time

    # Format and print the total elapsed time
    formatted_time = format_time(total_elapsed_time)
    print(f"Total elapsed time: {formatted_time}")

    return results_dict
# Call the parallel_attack function to initiate the parallelized attack
result_dict = parallel_attack(cipher)
concat = b''
# Print the results for each worker
for index in range(len(result_dict)):
    concat += result_dict[index]

b'\xf2\xa0mP\x7fF\x82\x85/\xf9\xd9h\x07F\x96\xed\xd3\xa1D\r\xee\xa1u9\x8c\xd9\xd8\xc6\xa3\x08g\xce\x0b\xfc7\xacC\x9ce"g\xab\x96\x18\xf0~`\xfa\xf3\xfdD\x94n\xa9\xc3j\xaf\xe1\x0f\xaa\xc3\xa2\x99\xba\x9e\x18+&5\x9f\xc0\x86q>n\x8e\xa0\xe3\x1e\x13\x04\x0b\xb0H\xb4\\\xd9,\x993\x19\xb0\x1c\xef\x08\x04\xdb\xc9\xa0\x95\x9a\x91F\xd3\xde\xfaB\x12\xcaoKi\x84\xfcc\xe6{\x87Q)\x0f\x9d\x1d\x99\xe7\xe0\x9b\xa4\x92\x99\xf6\xd3{}S\xf3\xfc\x904\xe1\xf8\x84e\xdf\x13\t\xf43#d*Cy\xf59\xd8\xe0\xc2$C\xab\xbc\xbd!\xdcL\xdc]:\xec.5{S\x173T1Lg\xa81T\x95B\xb3Z\x9bXzv\xce\x81\xcbj\xf2\xd1\xbc;\xba\x02\xec>4O\xb8\x90\xb8\xe3f9G\xfb\x82\xe4x\xcc\x96\xc2F\xc4~\xed=e\xefop\x9c\x8a\x84\xb4`\xaf\xfe\x1c\x11\xd0\xa2\r4\xc5}.\xc4\xf0\x12\xa0\x1e{\x0e\xcd\xdb\xed\x88\x81\x8d\x0e\xa3~\xd3\x0c\x1e\x84\xe2\x9d\xadE\x0b\xfdn\xb1\x05\x07_\xb0\xebA\xcc \xc6\xe3s\x97\x0f\x02\x87{A\xba\x1a\x0c\xf5B\xe0"\xbb2}\xc9Z<\xe1{\xadX{\'\x052\xfe\x94w\x80\x93\xe1MGFb\x19\xbb\xc1\x99!,\x97\xd9\xcb\xd4k\xd6\x16\x9am5\xb7\xa8\xcem{\x08S\xce\x04

NameError: ignored

In [ ]:
BLOCK_SIZE = 16
msg4 = bytearray(b"\t\t;stif rettif Asrennis hguohT\t\tuc rettuc A\nnis nniht dnA\t\t\t\t;st na dnA\nniht srerettops tfarcriapap dnA\t\t;stops tolb srettolb-re\t\trettis-ybab A\ntey reven ev'I\t\t\t\t\t-- stis-ybaB\ntel srettel daH\ten retto na tuB\nes rO\t\t\t.sto rev\n.to retto na ne(\nstab rettab A\ntacs srettacs rOhs gnittop A\n;)sgnittop rof s'def s'eno on tuB\n;b rednuob A\ndnuoa thguac rO\ndnuo\n.gnitto retto nniweL hplaR --\t\t")
msg4 = msg4[::-1]

part = lambda x, n: [x[i:i+16] for i in range(0, len(x), 16)]
msg4 = bytearray().join(part(msg4,BLOCK_SIZE)[::-1])

msgposta = msg4.decode('ascii')
print(msgposta)


A fitter fits;				Though sinners sin
A cutter cuts;				And thinners thin
And an aircraft spotter spots;		And paper-blotters blot
A baby-sitter				I've never yet
Baby-sits --				Had letters let
But an otter never ots.			Or seen an otter ot.

A batter bats
(Or scatters scats);
A potting shed's for potting;
But no one's found
A bounder bound
Or caught an otter otting.
		-- Ralph Lewin


In [ ]:
answer_server = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/padding-oracle/{email}/answer"
padding_ans = requests.post(answer_server,files = {'message': msgposta})
padding_ans.text

'¡Ganaste!\n'